In [ ]:
import functions.q_generator
import functions.data_preperation as dp


In [2]:
tickers = [
    # S&P 500 (10)
    "AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "NVDA", "JNJ", "V", "JPM",
    
    # Euro Stocks (5)
    "SAP.DE", "AIR.PA", "DAI.DE", "NESN.SW", "OR.PA",
    
    # FTSE 100 (5)
    "HSBA.L", "BP.L", "GSK.L", "VOD.L", "RIO.L",
    
    # Other Global Stocks (5)
    "TSM", "BABA", "SONY", "TCEHY", "TM"
]

num_tickers = len(tickers)



In [ ]:
data = dp.download_stock_data(tickers, years=5)

for t in tickers:
    data = dp.add_OC_CO_next_changes(data, t)

data = dp.remove_na(data)

dp.plot_tickers_pct_change_corr(data, tickers, title="Ticker % Change Correlation Heatmap")

In [3]:
def build_expressive_stock_circuit(
    tickers,
    correlations=None,
    qubits_per_stock=3,
    reps=1,
    entanglement_threshold=0.2,
    inter_entanglement_gate='cry',
):
    """
    Builds a scalable, expressive parameterized quantum circuit for stock modeling.

    Parameters:
        tickers (list[str]) :
            List of stock ticker names.
        correlations (np.ndarray or None) :
            Optional correlation matrix (symmetric, values in [-1, 1]).
            If None, no inter-stock entanglement is applied.
        qubits_per_stock (int) :
            Number of qubits per stock block (default: 3).
        reps (int) :
            Number of repetitions of the block (depth).
        entanglement_threshold (float) :
            Minimum absolute correlation to consider for inter-stock entanglement.
        inter_entanglement_gate (str) :
            Type of gate to use for inter-stock entanglement ('cry' or 'cz').

    Returns:
        QuantumCircuit :
            A fully parameterized and expressive quantum circuit.
    """
    num_stocks = len(tickers)
    total_qubits = num_stocks * qubits_per_stock
    qc = QuantumCircuit(total_qubits)

    # Assign qubit blocks per stock
    block_indices = [
        list(range(i * qubits_per_stock, (i + 1) * qubits_per_stock))
        for i in range(num_stocks)
    ]

    param_count = 0
    # Add expressive intra-stock blocks
    for ticker, qubits in zip(tickers, block_indices):
        num_params = reps * qubits_per_stock * 2  # 2 parameters (RY, RZ) per qubit per rep
        params = ParameterVector(f'{ticker}_Î¸', length=num_params)

        idx = 0
        for _ in range(reps):
            # Parameterized single-qubit rotations
            for q in qubits:
                qc.ry(params[idx], q)
                qc.rz(params[idx + 1], q)
                idx += 2

            # Entanglement (ring of CXs within stock block)
            for i in range(len(qubits)):
                qc.cx(qubits[i], qubits[(i + 1) % len(qubits)])

    # Add inter-stock entanglement (optional)
    if correlations is not None:
        assert correlations.shape == (num_stocks, num_stocks), "Correlation matrix must match number of stocks"

        for i in range(num_stocks):
            for j in range(i + 1, num_stocks):
                corr = correlations[i, j]
                if abs(corr) >= entanglement_threshold:
                    q1 = block_indices[i][qubits_per_stock // 2]  # middle qubit of block i
                    q2 = block_indices[j][qubits_per_stock // 2]  # middle qubit of block j
                    angle = abs(corr) * np.pi

                    if inter_entanglement_gate == 'cry':
                        qc.cry(angle, q1, q2)
                    elif inter_entanglement_gate == 'cz':
                        qc.cz(q1, q2)
                    else:
                        raise ValueError(f"Unsupported gate type: {inter_entanglement_gate}")

    return qc